In [1]:
import pandas as pd
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier)
from scipy.stats import randint as sp_randint
from sklearn.grid_search import RandomizedSearchCV
from sqlalchemy import create_engine
from xgboost.sklearn import XGBClassifier
import numpy as np
from sklearn import cross_validation

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
pd.set_option('max_columns',200)

In [4]:
df_dat = pd.read_csv(open('/data1/complete_file_1.csv','rU'), encoding='utf-8', engine='c')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,12,13,14,15,16,17,18,19,20,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4534763 entries, 0 to 4534762
Data columns (total 34 columns):
BrandName                        object
SupplyCountry                    object
HomeAwayFamily                   object
EmailId                          object
UserGuid                         object
FirstName                        object
LastName                         object
UserName                         object
EmailAddress                     object
CreateDate                       object
updateDate                       object
UserStatus                       object
About                            object
Employer                         object
Languages                        object
School                           object
Gender                           object
HomeTown                         object
RegisteredFlag                   object
SourceFacebookUserId             object
PublicProfileCompleteFlag        object
CompleteDate                     object
Current

In [6]:
df_dat = df_dat[df_dat['BrandName'] == 'VacationRentalsByOwner']

In [7]:
for i in df_dat.columns:
    print pd.value_counts(df_dat[i], dropna=False)

VacationRentalsByOwner    2520150
Name: BrandName, dtype: int64
United States    2520150
Name: SupplyCountry, dtype: int64
VRBO    2520150
Name: HomeAwayFamily, dtype: int64
13835832    435
64270906    111
63963677    109
14600264     54
96504704     53
92203996     49
3747717      48
63274201     43
7088326      42
30527729     40
4939867      40
4528436      37
67955181     37
6950353      36
3153585      30
74419768     28
88011622     27
41389676     25
40787628     25
83238332     25
8764936      24
95365411     23
74536657     23
4528437      22
5253070      22
90206708     22
19481870     22
75855997     21
53757969     20
2170053      20
           ... 
90777921      1
19474689      1
99166459      1
65611964      1
90777826      1
22650598      1
12250942      1
94972101      1
23668940      1
36251856      1
36251858      1
74000599      1
96163697      1
11086042      1
90777822      1
11086049      1
6891753       1
23668986      1
99166444      1
99166447      1
65612016  

In [8]:
feat_exist = ['About','Employer','Languages','School','HomeTown','SourceFacebookUserId','CurrentPublicProfilePictureId','SiteId']

In [9]:
for i in feat_exist:
    df_dat[i+'_exist'] = df_dat[i].apply(lambda x: 1 if pd.notnull(x) else 0)

In [10]:
Gender_cat = []
Gender_count = pd.value_counts(df_dat['Gender'])[:3]
Gender_count_dict = Gender_count.to_dict()
for i in Gender_count_dict:
    Gender_cat.append(i)
for j in Gender_cat:
    df_dat[j+'_Gender_cat'] = df_dat['Gender'].apply(lambda x: 1 if x == j else 0)

In [11]:
df_dat.RegisteredFlag = df_dat.RegisteredFlag.str.lower()

In [12]:
regflg_cat = []
regflg_count = pd.value_counts(df_dat['RegisteredFlag'])[:2]
regflg_count_dict = regflg_count.to_dict()
for i in regflg_count_dict:
    regflg_cat.append(i)
for j in regflg_cat:
    df_dat[j+'_regflg_cat'] = df_dat['RegisteredFlag'].apply(lambda x: 1 if x == j else 0)

In [13]:
df_all_dummy = pd.get_dummies(df_dat['ProfilePictureSource'], prefix='ProfPicSrc_')
df_dat = pd.concat((df_dat, df_all_dummy), axis=1)

In [14]:
df_all_dummy = pd.get_dummies(df_dat['DefaultProfilePictureFlag'], prefix='DefltProfPicFlg_')
df_dat = pd.concat((df_dat, df_all_dummy), axis=1)

In [15]:
df_all_dummy = pd.get_dummies(df_dat['PersonType'], prefix='PersonType_')
df_dat = pd.concat((df_dat, df_all_dummy), axis=1)

In [16]:
df_dat['AdultNum'] = df_dat['AdultNum'].interpolate()

In [17]:
filter_values = [0,2,4,6,8,10,14,18,22,30,40,50]

In [18]:
ANlabel = []
for i in range(len(filter_values)-1):
    ANlabel.append('AdultNum_'+str(filter_values[i])+'_to_'+str(filter_values[i+1]))

In [19]:
tef = pd.get_dummies(pd.cut(df_dat['AdultNum'], bins=filter_values,labels=ANlabel))

In [20]:
df_dat = pd.concat((df_dat, tef), axis=1)

In [21]:
df_dat['ChildrenNum'] = df_dat['ChildrenNum'].interpolate()

In [22]:
filter_values = [0,2,4,6,8,10,14,18,22,30]

In [23]:
ANlabel = []
for i in range(len(filter_values)-1):
    ANlabel.append('ChildrenNum_'+str(filter_values[i])+'_to_'+str(filter_values[i+1]))

In [24]:
ef = pd.get_dummies(pd.cut(df_dat['ChildrenNum'], bins=filter_values,labels=ANlabel))

In [25]:
df_dat = pd.concat((df_dat, ef), axis=1)

In [26]:
df_dat['ReservationStatus_target'] = df_dat['ReservationStatusType'].apply(lambda x: 1 if x =='CONFIRMED' else 0)

In [27]:
df_dat.Employer = df_dat.Employer.str.lower()

In [28]:
df_dat['Employer'] = df_dat.Employer.fillna('none')

In [29]:
df_dat['retired_'+'emp_cat'] = df_dat['Employer'].str.contains('retired').astype(int)

In [30]:
df_dat['employed_'+'emp_cat'] = df_dat['Employer'].str.contains('employed').astype(int)

In [31]:
df_dat['teacher_'+'emp_cat'] = df_dat['Employer'].str.contains('teacher').astype(int)

In [32]:
Language_cat = ['english','french','deutsch','italian','russian','spanish']

In [33]:
df_dat['Languages'] = df_dat.Languages.fillna('none')

In [34]:
for j in Language_cat:
    df_dat[j+'_lang_cat'] = df_dat['Languages'].str.contains(j).astype(int)

In [35]:
# df_dat = df_dat.iloc[:,34:]

In [36]:
date_feats = ['CreateDate','updateDate','CreatedDate'] #dates

In [37]:
date_feat = ['FirstLiveDate']

In [38]:
df_dat.FirstLiveDate = df_dat.FirstLiveDate.fillna(0)
df_dat.CreateDate = df_dat.CreateDate.fillna(0)
df_dat.updateDate = df_dat.updateDate.fillna(0)
df_dat.CreatedDate = df_dat.CreatedDate.fillna(0)

In [39]:
df_dat.FirstLiveDate = pd.to_datetime(df_dat.FirstLiveDate)
df_dat.CreateDate = pd.to_datetime(df_dat.CreateDate)
df_dat.CreatedDate = pd.to_datetime(df_dat.CreatedDate)
df_dat.updateDate = pd.to_datetime(df_dat.updateDate)

In [40]:
for i in date_feats:
    df_dat[i+'_day'] = df_dat[i].dt.day
    df_dat[i+'_dayofweek'] = df_dat[i].dt.dayofweek
    df_dat[i+'_dayofyear'] = df_dat[i].dt.dayofyear
    df_dat[i+'_hour'] = df_dat[i].dt.hour
    df_dat[i+'_minute'] = df_dat[i].dt.minute
    df_dat[i+'_month'] = df_dat[i].dt.month
    df_dat[i+'_quarter'] = df_dat[i].dt.quarter
    df_dat[i+'_week'] = df_dat[i].dt.week
    df_dat[i+'_year'] = df_dat[i].dt.year

In [41]:
for i in date_feat:
    df_dat[i+'_day'] = df_dat[i].dt.day
    df_dat[i+'_dayofweek'] = df_dat[i].dt.dayofweek
    df_dat[i+'_dayofyear'] = df_dat[i].dt.dayofyear
    df_dat[i+'_month'] = df_dat[i].dt.month
    df_dat[i+'_quarter'] = df_dat[i].dt.quarter
    df_dat[i+'_week'] = df_dat[i].dt.week
    df_dat[i+'_year'] = df_dat[i].dt.year

In [42]:
df_final = df_dat.iloc[:,34:]

In [43]:
df_final.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520150 entries, 0 to 4534762
Data columns (total 85 columns):
About_exist                            int64
Employer_exist                         int64
Languages_exist                        int64
School_exist                           int64
HomeTown_exist                         int64
SourceFacebookUserId_exist             int64
CurrentPublicProfilePictureId_exist    int64
SiteId_exist                           int64
M_Gender_cat                           int64
O_Gender_cat                           int64
F_Gender_cat                           int64
false_regflg_cat                       int64
true_regflg_cat                        int64
ProfPicSrc__facebook                   float64
ProfPicSrc__gravatar                   float64
ProfPicSrc__manual                     float64
DefltProfPicFlg__False                 float64
DefltProfPicFlg__True                  float64
PersonType__FRBO                       float64
PersonType__PM      

In [44]:
# df_dat[(df_dat['FirstLiveDate']).isnull()]

In [45]:
# for i in df_final.columns:
#     print pd.value_counts(pd.isnull(df_final[i]))

In [46]:
# df_dat[pd.isnull(df_dat['updateDate'])]['updateDate']

In [47]:
# pd.value_counts(df_dat[pd.isnull(df_dat['updateDate'])]['updateDate'])

In [48]:
for i in df_final.columns:
    df_final[i] = df_final[i].astype('int64')

In [49]:
df_final.to_csv('/data1/VRBO_data/df_all_brnds_wo_dates.csv')